 <img style="float: left;" src="https://docs.expert.ai/logo.png" width="150px">
 <img style="float: right;" src="https://i.postimg.cc/Hk7R2sK5/logo-chatbot-law.png" width="150px">
 
#  Natural Language Expert AI Lawyer

 **expert.ai Natural Language API v2** (https://developer.expert.ai/) parses and "understands" large volumes of text.

In this section we'll install and play with expert.ai Natural Language API to work with Python, and then introduce some concepts related to Natural Language Processing.

You can also download the source code of our Python SDK and this notebook from Github at https://github.com/therealexpertai/


## Working with NL API in Python
First you have to setup your account credentials; if you don't have them, get them at https://developer.expert.ai/ui/login

Set your environment variables with NL API credentials 

```bash
SET EAI_USERNAME=YOUR_USER
SET EAI_PASSWORD=YOUR_PASSWORD
```
or 

```bash
export EAI_USERNAME=YOUR_USER
export EAI_PASSWORD=YOUR_PASSWORD
```

as an alternative you can always add to your notebook the following statements

```python
import os
os.environ["EAI_USERNAME"] = 'YOUR_USER'
os.environ["EAI_PASSWORD"] = 'YOUR_PASSWORD'
```

Now let's play with Python and Natural Language Processing.

Currently the API supports five languages i.e. English, French, Spanish, Italian and German. You have to define the text you want to process and the language model to use for the analysis.

In [7]:
import os
import json
os.environ["EAI_USERNAME"] = 'xxxxxxxx'
os.environ["EAI_PASSWORD"] = 'xxxxx'

In [8]:
from expertai.nlapi.cloud.client import ExpertAiClient
client = ExpertAiClient()

In [9]:
language= 'en'

<img style="float: right;" src="https://i.postimg.cc/Hk7R2sK5/logo-chatbot-law.png" width="150px">

# Expertai  Lawyer analysis 
Expertai analysis function call 3 services of expertai api and transform text to json :
* Tokenization & Lemmatization
* Key Elements
* Classification

In [10]:
def expertai_analysis(text):
    document = client.specific_resource_analysis(
    body={"document": {"text": text}}, 
    params={'language': language, 'resource': 'disambiguation'})
    lemma_data = [token.lemma for token in document.tokens]
    
    document = client.specific_resource_analysis(
    body={"document": {"text": text}}, 
    params={'language': language, 'resource': 'relevants'})
    mainlemma_data =[{"lemma":mainlemma.value,"score":mainlemma.score} for mainlemma in document.main_lemmas]
    
    iptc_classification = client.classification(
    body={"document": {"text": text}},
    params={'taxonomy': 'iptc', 'language': language})
    classification_data = [{"category":category.label,"id":category.id_,"frequency":category.frequency} for category in iptc_classification.categories]
    
    data = {"text":text ,"data_expertai":{"lemma":lemma_data,"mainlemma":mainlemma_data,"classification":classification_data}}
    
    return data

### Example of output function of expertai analysis

In [11]:
question_example = """Can I apply for citizenship even though I have a DUI on my background ?"""

In [12]:
print(json.dumps(expertai_analysis(question_example), indent=4))

{
    "text": "Can I apply for citizenship even though I have a DUI on my background ?",
    "data_expertai": {
        "lemma": [
            "Can",
            "I",
            "apply",
            "for",
            "citizenship",
            "even though",
            "I",
            "have",
            "a",
            "DUI",
            "on",
            "my",
            "background",
            "?"
        ],
        "mainlemma": [
            {
                "lemma": "DUI",
                "score": 46.9
            },
            {
                "lemma": "citizenship",
                "score": 34.2
            },
            {
                "lemma": "background",
                "score": 18.7
            }
        ],
        "classification": [
            {
                "category": "Immigration",
                "id": "20000771",
                "frequency": 100.0
            }
        ]
    }
}


<img style="float: right;" src="https://i.postimg.cc/Hk7R2sK5/logo-chatbot-law.png" width="150px">

# Expertai Lawyer matching
This function use the output of analysis function to match the question with the best answer, the score of matching is based :
* Score of Classifcation
* Score of main lemma

In [13]:
def expertai_matching(answers,question):
    id_inc=0
    rank_answers=[]
    analyse_question = expertai_analysis(question)
    get_classification_question = [classification_question['category'] for classification_question in analyse_question['data_expertai']['classification']]
    get_mainlemma_question = [mainlemma_question['lemma'] for mainlemma_question in analyse_question['data_expertai']['mainlemma']]
    for answer in answers:
        answer['id']=id_inc
        get_classification_answer = [classification_answer['category'] for classification_answer in answer['data_expertai']['classification']]
        get_mainlemma_answer = [mainlemma_answer['lemma'] for mainlemma_answer in answer['data_expertai']['mainlemma']]
        
        matching_classification = list(set(get_classification_question).intersection(set(get_classification_answer)))
        matching_mainlemma = list(set(get_mainlemma_question).intersection(set(get_mainlemma_answer)))
        
        score_matching = len(matching_classification) + len(matching_mainlemma)
        rank_answers.append([id_inc,score_matching])
        id_inc=id_inc+1
        
    sort_rank_id = sorted(rank_answers, key = lambda x: x[1],reverse=True)
    best_rank_id = sort_rank_id[0]
    if best_rank_id[1]>0:
        best_answer = next(answer for answer in answers if answer["id"] == best_rank_id[0])
        return best_answer['text']
    else :
        return "Sorry we can't answer to your question"

### Example of output function of expertai matching

Load prepared data : answers lawyers 

In [14]:
with open('data/answers_expertai_analysis.json') as f:
    answers_expertai_analysis = json.load(f)

In [15]:
question_example_a = """I wanna change a Tourist Visa to a Student Visa, how i can do that ?"""

In [16]:
expertai_matching(answers_expertai_analysis,question_example_a)

'Tourist visas or B-2 visas can change to student or F-1 visas with the right circumstances and paperwork filled out through the United States Citizenship and Immigration Services or the USCIS. Getting approval from the requested documentation does not come with any guarantee that the tourist can remain in the country and go to school.\n'

In [17]:
question_example_b = """Can I apply for citizenship even though I have a DUI on my background ?"""

In [18]:
expertai_matching(answers_expertai_analysis,question_example_b)

'It seems unlikely that a single DUI would negatively impact your application for citizenship. Applicants for citizenship must show 5 years of good moral character, so if the DUI was in the last five years you might want to wait until more than five years have passed. You will also want to make sure that the case has been closed and all obligations associated with it satisfied.\n'

<img style="float: right;" src="https://i.postimg.cc/Hk7R2sK5/logo-chatbot-law.png" width="150px">

# It's your turn let's try Expertai Lawyer

You can ask about :
* Immigration Law
* Criminal Law
* DUI / DWI

In [ ]:
expertai_matching(answers_expertai_analysis,"let's try your question with expert ai lawyer ....")